#install

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import json

# URL

In [ ]:
url = 'https://goods.musinsa.com/api/goods/v2/review/total-count?goodsNo=2343198&selectedSimilarNo=0'

headers = {
  
    'referer': 'https://store.musinsa.com/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36 Edg/100.0.1185.29'}
resp = requests.get(url, headers = headers)
resp.text

'{"data":{"totalCount":589,"styleCount":133,"photoCount":133,"goodsCount":323,"eventCount":0,"eventTotalCount":0}}'

In [ ]:
dic_resp = json.loads(resp.text)
reviews_num = dic_resp['data']['totalCount']
reviews_num

589

# 크롤링

In [ ]:
items_num = []
items_name = []

for page in range(3):
  page += 1
  url = 'https://www.musinsa.com/ranking/best?period=month_3&age=ALL&mainCategory=&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page='+str(page)+'&viewType=small&priceMin=&priceMax=#pol2398497'
  resp = requests.get(url)
  soup = BeautifulSoup(resp.content, 'lxml')
  
  numbers = soup.select('#goodsRankList > li')
  names = soup.select('#goodsRankList > li > div.li_inner > div.article_info > p.list_info > a')
  
  for number, name in tqdm(zip(numbers, names)):
    items_num.append(number['data-goods-no'])
    items_name.append(name['title'])

len(items_name)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

270

In [ ]:
review_info = {
    '상품번호' : [],
    '상품명' : [],
    '후기' : []
}


for item_num, item_name in tqdm(zip(items_num, items_name)):
  url = 'https://goods.musinsa.com/api/goods/v2/review/total-count?goodsNo='+str(item_num)+'&selectedSimilarNo=0'

  headers = {
      'referer': 'https://store.musinsa.com/',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36 Edg/100.0.1185.29'}
  resp = requests.get(url, headers = headers)
  resp.text
  dic_resp = json.loads(resp.text)
  reviews_num = dic_resp['data']['goodsCount']
  
  review_info['상품번호'].append(item_num)
  review_info['상품명'].append(item_name)
  review_info['후기'].append(reviews_num)




review_info


# 시각화

In [ ]:
df = pd.DataFrame(review_info)
df

,상품번호,상품명,후기
0,2043036,[송필드 PICK][SET] 헤비웨이트 CGP 아치 로고 트레이닝 셋업_멜란지그레이,2135
1,2343198,Ray Yacht Parka Black,315
2,1139192,화란 세미오버 가디건 블랙,5340
3,1571879,[SET] YKK 2-WAY 테크 숏 자켓 셋업_그레이,2747
4,1778404,2WAY 스웻 후드 집업 (MELANGE GREY),21814
...,...,...,...
265,2200194,[패키지] 투 파인더 세트 5종 (후드집업 JUHD4453 + 트레이닝팬츠 JULP...,97
266,883004,아디브레이크 팬츠 - 블랙 / GN2807,5039
267,1224095,세미 와이드 히든 밴딩 슬랙스 [미디엄 그레이],11154
268,2032901,릴렉스드 테이퍼드 히든 밴딩 크롭 슬랙스 [블랙],2387


In [ ]:
users_info = {
    '성별' : [],
    '키' : [],
    '몸무게' : [],
    '고객레벨' : [],
    '구매한 사이즈' :[]
}